## Assignment 2
#### Group Members:
- Abhijit Anil (aa94375)
- Anubhav Nehru (an33868)
- Antra Sinha (as224589)
- Avi Walyat(aw39578)
- Gaytri Riya Vasal(grv377)

#### Problem Description

Building a Crowdsourced Recommender System
High level description: The objective of this group assignment is to create the building blocks of a crowdsourced recommender system. It should accept user inputs in the form of desired attributes of a product and come up with 3 recommendations.
Obtain reviews of craft beers from beeradvocate.com. I would suggest using the following link, which shows the top 250 beers sorted by ratings:
https://www.beeradvocate.com/beer/top-rated/
The nice feature of the above link is that it is a single-page listing of 250 top-rated beers (avoids the pagination feature, which you need in cases where listings go on for many pages). The way beeradvocate.com organizes reviews is that it provides about 25 reviews per page. The output file should have 3 columns: product_name, product_review, and user_rating.




## Task A

#### Extract about 5-6k reviews


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Sample HTML snippet
url = "https://www.beeradvocate.com/beer/top-rated/"
response = requests.get(url)

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

a_tags = soup.find_all('a', href=True)

beer_profiles = []

# Extract and print the 'href' attribute values for all <a> elements
for a_tag in a_tags:
    pattern = r'/beer/profile/[^/]+/[^/]+/'
    href_link = a_tag['href']

    # Find all matches in the HTML content
    if re.match(pattern, href_link):
        beer_profiles.append('https://www.beeradvocate.com'+ href_link)


print(href_link)

/beer/top-rated/


In [2]:
pages = [0,20,40,60]
beer_links = []
for beers in beer_profiles:
    for j in pages:
        # print(j)
        beer_links.append(beers + '?view=beer&show=recent&start='+str(j)+'#lists')

beer_links

['https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=0#lists',
 'https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=20#lists',
 'https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=40#lists',
 'https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=60#lists',
 'https://www.beeradvocate.com/beer/profile/23222/113831/?view=beer&show=recent&start=0#lists',
 'https://www.beeradvocate.com/beer/profile/23222/113831/?view=beer&show=recent&start=20#lists',
 'https://www.beeradvocate.com/beer/profile/23222/113831/?view=beer&show=recent&start=40#lists',
 'https://www.beeradvocate.com/beer/profile/23222/113831/?view=beer&show=recent&start=60#lists',
 'https://www.beeradvocate.com/beer/profile/26/42349/?view=beer&show=recent&start=0#lists',
 'https://www.beeradvocate.com/beer/profile/26/42349/?view=beer&show=recent&start=20#lists',
 'https://www.beeradvocate.com/beer/profile/2

In [3]:
!pip install selenium

In [4]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
#WebDriver is a browser automation framework that works with open source APIs.
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(options=chrome_options)
wd.get("https://www.beeradvocate.com/community/login/")

from selenium.common.exceptions import TimeoutException

wd.maximize_window()

while True:
    try:
        WebDriverWait(wd, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'form#pageLogin input#ctrl_pageLogin_login'))).send_keys("Beer_Lover_4")
        print("entered username")
        wd.find_element(By.CSS_SELECTOR,'form#pageLogin input#ctrl_pageLogin_password').send_keys('x+Z}K,*24x!9NG7')
        print("entered password")
        wd.find_element(By.CSS_SELECTOR,'form#pageLogin input[type=submit]').click()
        print("logged in")
        break
    except TimeoutException:
        print("too much time",TimeoutException)


entered username
entered password
logged in


In [5]:
from selenium.common.exceptions import NoSuchElementException
beer_names = []
beer_companies = []
users = []
users_score = []
users_rating = []
users_comment = []
pages = [0, 20, 40, 60]
for beer_link in reversed(beer_links):

    response = requests.get(beer_link)
    if response.status_code == 200:
    # Page exists, continue with Selenium actions
        wd.get(beer_link)

        beer_name_html = wd.find_element(
            by=By.XPATH,
            value="/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/h1",
        )
        beer_name = beer_name_html.text
        print(beer_name)
        beer_company_html = wd.find_element(
            by=By.XPATH,
            value="/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/h1/span",
        )
        beer_company = beer_company_html.text
        for i in range(27):
            user_html = ""
            try:
                user_html = wd.find_element(
                    by=By.XPATH,
                    value="/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[4]/div[10]/div/div["
                    + str(i)
                    + "]/div[2]/span[1]/b[2]/a",
                )
                users.append(user_html.text)
                # print(i)
                user_score_html = wd.find_element(
                    by=By.XPATH,
                    value="/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[4]/div[10]/div/div["
                    + str(i)
                    + "]/div[2]/span[1]/b[1]",
                )
                users_score.append(user_score_html.text)
                # print(i)
                users_rating.append(" ")
                users_comment.append(" ")
                beer_names.append(beer_name)
                beer_companies.append(beer_company)
            except NoSuchElementException:
                try:
                    user_html = wd.find_element(
                        by=By.XPATH,
                        value="/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[4]/div[10]/div/div["
                        + str(i)
                        + "]/div[2]/span[1]/b/a",
                    )
                    users.append(user_html.text)
                    user_score_html = wd.find_element(
                        by=By.XPATH,
                        value="/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[4]/div[10]/div/div["
                        + str(i)
                        + "]/div[2]/span[2]",
                    )
                    users_score.append(user_score_html.text)
                    users_rating_html = wd.find_element(
                        by=By.XPATH,
                        value="/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[4]/div[10]/div/div["
                        + str(i)
                        + "]/div[2]/span[5]",
                    )
                    users_rating.append(users_rating_html.text)
                    user_comment_html = wd.find_element(
                        by=By.XPATH,
                        value="/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[4]/div[10]/div/div["
                        + str(i)
                        + "]/div[2]/div",
                    )
                    users_comment.append(user_comment_html.text)
                    beer_names.append(beer_name)
                    beer_companies.append(beer_company)
                except NoSuchElementException:
                    test = 0


Stickee Monkee
Firestone Walker Brewing Co.
Stickee Monkee
Firestone Walker Brewing Co.
Stickee Monkee
Firestone Walker Brewing Co.
Stickee Monkee
Firestone Walker Brewing Co.
Abraxas
Perennial Artisan Ales
Abraxas
Perennial Artisan Ales
Abraxas
Perennial Artisan Ales
Abraxas
Perennial Artisan Ales
All That Is And All That Ever Will Be
Tree House Brewing Company
All That Is And All That Ever Will Be
Tree House Brewing Company
All That Is And All That Ever Will Be
Tree House Brewing Company
All That Is And All That Ever Will Be
Tree House Brewing Company
The Abyss
Deschutes Brewery
The Abyss
Deschutes Brewery
The Abyss
Deschutes Brewery
The Abyss
Deschutes Brewery
Abt 12
Brouwerij St. Bernardus NV


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: headless chrome=117.0.5938.132)
Stacktrace:
0   chromedriver                        0x0000000100b62d98 chromedriver + 4337048
1   chromedriver                        0x0000000100b5ae14 chromedriver + 4304404
2   chromedriver                        0x0000000100787a5c chromedriver + 293468
3   chromedriver                        0x000000010076fe68 chromedriver + 196200
4   chromedriver                        0x000000010076fc6c chromedriver + 195692
5   chromedriver                        0x000000010076e0b8 chromedriver + 188600
6   chromedriver                        0x000000010076ebf4 chromedriver + 191476
7   chromedriver                        0x000000010078a4b8 chromedriver + 304312
8   chromedriver                        0x0000000100807f60 chromedriver + 819040
9   chromedriver                        0x0

In [ ]:
comment_list = pd.DataFrame({ "Beer":beer_names, "Beer Company": beer_companies, "User Name": users, "Score" : users_score, "Rating": users_rating, "Comment": users_comment})
comment_list

In [ ]:
comment_list.to_csv('output_5.csv', index=    False)
wd.quit()

We started to collect the unstructured data by scrapping ~19K reviews. We filtered the reviews based beer with atleast 15 reviews and no empty comment in the reviews, which resulted in a total of ~5K reviews


## Task B

Assume that a customer, who will be using this recommender system, has specified 3 attributes in a product. E.g., one website describes multiple attributes of beer (but you should choose attributes from the actual data)
https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/
- Aggressive (Boldly assertive aroma and/or taste)
- Balanced: Malt and hops in similar proportions; equal representation of malt sweetness and hop bitterness in the flavor — especially at the finish
- Complex: Multidimensional; many flavors and sensations on the palate
- Crisp: Highly carbonated; effervescent
- Fruity: Flavors reminiscent of various fruits or Hoppy: Herbal, earthy, spicy, or citric aromas and flavors of hops or Malty: Grainy, caramel-like; can be sweet or dry
- Robust: Rich and full-bodied

Use the above attributes as examples only, for a word frequency analysis of beer reviews is a better way to find important attributes in the actual data.
Assume that a customer has specified 3 attributes of the product as being important to him or her.


In [6]:
import pandas as pd
import numpy as np
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import warnings
warnings.filterwarnings("ignore")

In [8]:
df = pd.read_csv("output_5.csv")
df.head()

,Beer,Beer Company,User Name,Score,Rating,Comment
0,Samuel Adams Utopias\nBoston Beer Company (Sam...,Boston Beer Company (Samuel Adams),JesseLara,4.84,,
1,Samuel Adams Utopias\nBoston Beer Company (Sam...,Boston Beer Company (Samuel Adams),Mjtucci,4.97,look: 4.5 | smell: 5 | taste: 5 | feel: 5 | ov...,"Unique\nBrown color\nSweet malty, maple flavor..."
2,Samuel Adams Utopias\nBoston Beer Company (Sam...,Boston Beer Company (Samuel Adams),pancake,4.77,,
3,Samuel Adams Utopias\nBoston Beer Company (Sam...,Boston Beer Company (Samuel Adams),Ziosa,4.75,,
4,Samuel Adams Utopias\nBoston Beer Company (Sam...,Boston Beer Company (Samuel Adams),Nedster10,4.50,,


In [9]:
df2 = df.loc[df['Comment']!=' '][['Beer', 'Score', 'Comment']]

df2['Comment'] = df2['Comment'].map(lambda x: re.sub(r'[^\w\s]', '', x.replace("\n", " ")))
df2['Beer'] = df2['Beer'].str.split('\n').str[0]


#Filter for brands with greater than 15 reviews
x = (df2[['Beer']].value_counts() > 15).reset_index()

df3  = df2[df2['Beer'].isin(x[x[0] == True]['Beer'])]

df3.columns = ['product_name' , 'user_rating' , 'product_review']
df3.head()

,product_name,user_rating,product_review
1,Samuel Adams Utopias,4.97,Unique Brown color Sweet malty maple flavor Ma...
8,Samuel Adams Utopias,4.71,Its the Port wine of the beer world well sort ...
9,Samuel Adams Utopias,3.93,Sample from bottle Cardinal Pub Bar Stavanger...
11,Samuel Adams Utopias,4.44,Ive only had this at Extreme Beer Fest I would...
21,Samuel Adams Utopias,4.43,2021 vintage much more complex than my previou...


In [10]:
df = df3

In [11]:
# Assuming you have a DataFrame 'comment_list' with a column "Comment" containing the text data
data = df['product_review']

all_words = []

# Tokenize each comment into words and append them to the 'all_words' list
for comment in data:
    if pd.notna(comment):
        words = comment.lower().split()
        all_words.extend(words)




# Calculate word frequencies
word_freq = Counter(all_words)

# Get the top N most common words (adjust N as needed)
N = 100
top_N_words = word_freq.most_common(N)

In [12]:
# Create a DataFrame with two columns
top_100_df = pd.DataFrame(top_N_words, columns=['Word', 'Freq'])
top_100_df.head(20)

,Word,Freq
0,the,12883
1,a,12678
2,and,10947
3,of,7822
4,is,5632
5,with,5617
6,to,4225
7,this,3762
8,i,3329
9,it,3154


In [13]:
df.shape

(3544, 3)

In [14]:
# Dropping NA values
df = df[pd.notna(df['product_review'])]

In [15]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def comment_preprocessing(comment) :

    # Converting coomments to lower case, tokenizing the comments and removing punctuation from a comments
    words = [re.sub(r'[^a-zA-Z0-9]', '', i) for i in word_tokenize(comment.lower())]
    words = [word for word in words if word not in stop_words and word]
    return (words,' '.join(words))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/antrasinha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/antrasinha/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/antrasinha/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/antrasinha/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/antrasinha/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/antrasinha/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    |

True

In [17]:
# Initialize empty lists to collect the results
word_lists = []
cleaned_sentences = []

for comment in df['product_review']:
    if pd.notna(comment) and isinstance(comment, str):
        output = comment_preprocessing(comment)
        word_lists.append(output[0])
        cleaned_sentences.append(output[1])

# After processing all comments, assign the lists to DataFrame columns
df['Word_List'] = word_lists
df['Cleaned_Sentence'] = cleaned_sentences

In [18]:
all_words = []
# Assuming 'Word_List' is the column containing lists of words in your DataFrame
for i in df['Word_List']:
    all_words.extend(i)

# Calculate word frequencies
word_freq = Counter(all_words)
# Get the top N most common words (adjust N as needed)
N = 100
top_N_words = word_freq.most_common(N)

# Create a DataFrame with two columns
top_100_df = pd.DataFrame(top_N_words, columns=['Word', 'Freq'])
top_100_df.head(20)

,Word,Freq
0,beer,2513
1,head,1971
2,taste,1718
3,dark,1482
4,chocolate,1394
5,sweet,1160
6,notes,1158
7,like,1153
8,bourbon,1102
9,coffee,1074


In [19]:
selected_attributes = ['light', 'dark', 'carbonation']

After performing word frequency analysis on the reviews present in the dataset and gaining insights for the website(https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/) about attributes of a beer, we selected light, dark and carbonation as our final three selected attributes of the product as being important to a customer. Light and dark beer may give a comaprison between how strong the beer is and the carbonation will give attribute about how gassy the beer is and all these 3 attributes may impact it's style and strength



## Task C

Perform a similarity analysis using cosine similarity (without word embeddings) with the 3 attributes specified by the customer and the reviews.

The similarity script should accept as input a file with the product attributes, and calculate similarity scores (between 0 and 1) between these attributes and each review. That is, the output file should have 3 columns – product_name (for each product, the product_name will repeat as many times as there are reviews of the product), product_review and similarity_score.


In [20]:
clean_df = df.copy(deep=True)

In [21]:
cleaned_comments = []
cleaned_comments = [i for i in df['Cleaned_Sentence']]

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(comments, attributes):
    # Initialize the Count Vectorizer
    count_vectorizer = CountVectorizer()

    # Fit and transform the comments and attributes
    count_matrix = count_vectorizer.fit_transform(comments + [" ".join(attributes)])

    # Calculate cosine similarity between comments and attributes
    similarity_matrix = cosine_similarity(count_matrix)

    # Extract the similarity scores for comments vs. attributes
    comment_similarity = similarity_matrix[:len(comments), len(comments):]

    return comment_similarity


similarity_matrix = calculate_cosine_similarity(cleaned_comments,selected_attributes)
#print(similarity_matrix)


In [23]:
count_vectorizer = CountVectorizer()

def cosine_bow(data):
    final_data=[data,required_attributes]
    vector_matrix = count_vectorizer.fit_transform(final_data)
    tokens = count_vectorizer.get_feature_names_out()
    cosine_similarity_matrix = cosine_similarity(vector_matrix)
    return cosine_similarity_matrix[0][1]

In [24]:
clean_df

,product_name,user_rating,product_review,Word_List,Cleaned_Sentence
1,Samuel Adams Utopias,4.97,Unique Brown color Sweet malty maple flavor Ma...,"[unique, brown, color, sweet, malty, maple, fl...",unique brown color sweet malty maple flavor ma...
8,Samuel Adams Utopias,4.71,Its the Port wine of the beer world well sort ...,"[port, wine, beer, world, well, sort, drinks, ...",port wine beer world well sort drinks like liq...
9,Samuel Adams Utopias,3.93,Sample from bottle Cardinal Pub Bar Stavanger...,"[sample, bottle, cardinal, pub, bar, stavanger...",sample bottle cardinal pub bar stavanger beaut...
11,Samuel Adams Utopias,4.44,Ive only had this at Extreme Beer Fest I would...,"[ive, extreme, beer, fest, wouldnt, usually, g...",ive extreme beer fest wouldnt usually go somet...
21,Samuel Adams Utopias,4.43,2021 vintage much more complex than my previou...,"[2021, vintage, much, complex, previous, vinta...",2021 vintage much complex previous vintages be...
...,...,...,...,...,...
19828,Kentucky Brunch Brand Stout,5.00,Amazing stout expensive but worth the price,"[amazing, stout, expensive, worth, price]",amazing stout expensive worth price
19831,Kentucky Brunch Brand Stout,5.00,This beer meet and exceeded all the unreal hyp...,"[beer, meet, exceeded, unreal, hype, simply, a...",beer meet exceeded unreal hype simply amazing
19833,Kentucky Brunch Brand Stout,5.00,Let it sit and warm to the room a little unbe...,"[let, sit, warm, room, little, unbelievable]",let sit warm room little unbelievable
19834,Kentucky Brunch Brand Stout,4.67,A small pour ebony dark no real head Intense r...,"[small, pour, ebony, dark, real, head, intense...",small pour ebony dark real head intense rich a...


In [25]:
results_df=clean_df.copy()

results_df["cosine_similarity"]=similarity_matrix

results_df[['product_name', 'product_review' , 'cosine_similarity']].sort_values(by = 'cosine_similarity' , ascending= False).head()

,product_name,product_review,cosine_similarity
11067,Black Tuesday,From the 750 ml bottle in a snifter via Blueja...,0.455016
450,Abt 12,Look Dark red when help to the light huge li...,0.439941
4386,Clare's Thirsty Ale,Pours black with a finger of mocha head Smells...,0.433013
18192,A Deal With The Devil - Double Oak-Aged,2017 vintage Dark brown black with oily appear...,0.428845
700,Darkstar November,A Pour jet black with a thin tan head Light s...,0.425628


In [26]:
results_df.groupby("product_name")["cosine_similarity"].mean().sort_values(ascending=False)[:3] # top3 recommendations in terms

product_name
Darkness               0.136109
BDCS                   0.123605
Hold On To Sunshine    0.120915
Name: cosine_similarity, dtype: float64


## Task D

For every review, perform a sentiment analysis (using VADER or any LLM). In case you have to change the default values of words in the VADER lexicon, use this article: https://medium.com/swlh/adding-context-to-unsupervised-sentiment-analysis-7b6693d2c9f8


In [27]:
import nltk
import pandas as pd

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

#Convert the top-10 attributes to score 0
for word in top_100_df[:10] : 
  analyzer.lexicon[word] = 0

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/antrasinha/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [28]:
#function for calculating sentiment score
def sentiment_scores(review):
    score = analyzer.polarity_scores(review)['compound']
    return score

In [29]:
# Create a function to analyze sentiment
def analyze_sentiment(comment):
    sentiment = analyzer.polarity_scores(comment)
    return sentiment

# Apply the sentiment analysis function to each row in the DataFrame
results_df["sentiment"] = results_df["Cleaned_Sentence"].apply(analyze_sentiment)
# Extract the compound sentiment score
results_df['sentiment_score'] = results_df['sentiment'].apply(lambda x: x['compound'])


In [30]:
results_df.head()

,product_name,user_rating,product_review,Word_List,Cleaned_Sentence,cosine_similarity,sentiment,sentiment_score
1,Samuel Adams Utopias,4.97,Unique Brown color Sweet malty maple flavor Ma...,"[unique, brown, color, sweet, malty, maple, fl...",unique brown color sweet malty maple flavor ma...,0.00000,"{'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'comp...",0.4588
8,Samuel Adams Utopias,4.71,Its the Port wine of the beer world well sort ...,"[port, wine, beer, world, well, sort, drinks, ...",port wine beer world well sort drinks like liq...,0.09759,"{'neg': 0.0, 'neu': 0.749, 'pos': 0.251, 'comp...",0.8271
9,Samuel Adams Utopias,3.93,Sample from bottle Cardinal Pub Bar Stavanger...,"[sample, bottle, cardinal, pub, bar, stavanger...",sample bottle cardinal pub bar stavanger beaut...,0.18490,"{'neg': 0.057, 'neu': 0.716, 'pos': 0.227, 'co...",0.7269
11,Samuel Adams Utopias,4.44,Ive only had this at Extreme Beer Fest I would...,"[ive, extreme, beer, fest, wouldnt, usually, g...",ive extreme beer fest wouldnt usually go somet...,0.00000,"{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'comp...",0.5859
21,Samuel Adams Utopias,4.43,2021 vintage much more complex than my previou...,"[2021, vintage, much, complex, previous, vinta...",2021 vintage much complex previous vintages be...,0.00000,"{'neg': 0.0, 'neu': 0.775, 'pos': 0.225, 'comp...",0.4404


We used Vader to perform a sentiment analysis and inorder to ensure that the attributes do not affect the sentiment score themselves, we manually set the top 10 attributes score to 0 by altering the lexicon

## Task E

Create an evaluation score for each beer that uses both similarity and sentiment scores.


Now recommend 3 products to the customer.


In [31]:
results_df["eval_score"] = results_df["cosine_similarity"]*results_df["sentiment_score"]
results_df.head()

,product_name,user_rating,product_review,Word_List,Cleaned_Sentence,cosine_similarity,sentiment,sentiment_score,eval_score
1,Samuel Adams Utopias,4.97,Unique Brown color Sweet malty maple flavor Ma...,"[unique, brown, color, sweet, malty, maple, fl...",unique brown color sweet malty maple flavor ma...,0.00000,"{'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'comp...",0.4588,0.000000
8,Samuel Adams Utopias,4.71,Its the Port wine of the beer world well sort ...,"[port, wine, beer, world, well, sort, drinks, ...",port wine beer world well sort drinks like liq...,0.09759,"{'neg': 0.0, 'neu': 0.749, 'pos': 0.251, 'comp...",0.8271,0.080717
9,Samuel Adams Utopias,3.93,Sample from bottle Cardinal Pub Bar Stavanger...,"[sample, bottle, cardinal, pub, bar, stavanger...",sample bottle cardinal pub bar stavanger beaut...,0.18490,"{'neg': 0.057, 'neu': 0.716, 'pos': 0.227, 'co...",0.7269,0.134404
11,Samuel Adams Utopias,4.44,Ive only had this at Extreme Beer Fest I would...,"[ive, extreme, beer, fest, wouldnt, usually, g...",ive extreme beer fest wouldnt usually go somet...,0.00000,"{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'comp...",0.5859,0.000000
21,Samuel Adams Utopias,4.43,2021 vintage much more complex than my previou...,"[2021, vintage, much, complex, previous, vinta...",2021 vintage much complex previous vintages be...,0.00000,"{'neg': 0.0, 'neu': 0.775, 'pos': 0.225, 'comp...",0.4404,0.000000


In [32]:
beer_reco_similarity = results_df.groupby("product_name").mean().sort_values(by='eval_score',ascending=False).reset_index() #top3 recommendations in terms of cosine similarity


In [33]:
beer_reco_similarity[:3]

,product_name,user_rating,cosine_similarity,sentiment_score,eval_score
0,Darkness,4.484865,0.136109,0.779189,0.114307
1,BDCS,4.395556,0.123605,0.839330,0.110087
2,Everett,4.572632,0.114460,0.876179,0.106064


Darkness, BDCS, and Everett are recommended to the customer. These beers have the top three evaluation scores of 0.114307, 0.110087, and 0.106064. The following analysis has deemed the evaluation score a valuable metric and combination of similarity and sentiment scores. Sentiment scores measure how frequently customers utilize positive and negative language in association with the product - As a result, a higher sentiment score is likely indicative of a positive customer perception of the product. Likewise, similarity scores demonstrate the specific percentage of the text contained within the comparison database. In this scenario, greater similarity scores represent a strongly associated match between user preferences and products, implying higher customer satisfaction, trust, etc.

## Task F

How would your recommendation change if you use word vectors (e.g., the spaCy package with medium sized pretrained word vectors) instead of the plain vanilla bag-of-words cosine similarity? One way to analyze the difference would be to consider the % of reviews that mention a preferred attribute. E.g., if you recommend a product, what % of its reviews mention an attribute specified by the customer? Do you see any difference across bag-of-words and word vector approaches? Explain. This article may be useful: https://medium.com/swlh/word-embeddings-versus-bag-of-words-the-curious-case-of-recommender-systems-6ac1604d4424?source=friends_link&sk=d746da9f094d1222a35519387afc6338
Note that the article doesn’t claim that bag-of-words will always be better than word embeddings for recommender systems. It lays out conditions under which it is likely to be the case. That is, depending on the attributes you use, you may or may not see the same effect.


In [34]:
!pip install spacy -q

In [35]:
import spacy

!python -m spacy download en_core_web_sm -q

!python -m spacy download en_core_web_md -q

!python -m spacy download en_core_web_lg -q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [36]:
required_attributes = ' '.join(selected_attributes)

In [39]:
def word_embeddings(review):
    review_nlp = nlp(review)
    score = review_nlp.similarity(attributes_nlp)
    return score

nlp = spacy.load('en_core_web_md')
attributes_nlp = nlp(required_attributes)
results_df['word_embedding_similarity'] = results_df['Cleaned_Sentence'].apply(word_embeddings)
results_df.head()

,product_name,user_rating,product_review,Word_List,Cleaned_Sentence,cosine_similarity,sentiment,sentiment_score,eval_score,word_embedding_similarity
1,Samuel Adams Utopias,4.97,Unique Brown color Sweet malty maple flavor Ma...,"[unique, brown, color, sweet, malty, maple, fl...",unique brown color sweet malty maple flavor ma...,0.00000,"{'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'comp...",0.4588,0.000000,0.573489
8,Samuel Adams Utopias,4.71,Its the Port wine of the beer world well sort ...,"[port, wine, beer, world, well, sort, drinks, ...",port wine beer world well sort drinks like liq...,0.09759,"{'neg': 0.0, 'neu': 0.749, 'pos': 0.251, 'comp...",0.8271,0.080717,0.511847
9,Samuel Adams Utopias,3.93,Sample from bottle Cardinal Pub Bar Stavanger...,"[sample, bottle, cardinal, pub, bar, stavanger...",sample bottle cardinal pub bar stavanger beaut...,0.18490,"{'neg': 0.057, 'neu': 0.716, 'pos': 0.227, 'co...",0.7269,0.134404,0.720679
11,Samuel Adams Utopias,4.44,Ive only had this at Extreme Beer Fest I would...,"[ive, extreme, beer, fest, wouldnt, usually, g...",ive extreme beer fest wouldnt usually go somet...,0.00000,"{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'comp...",0.5859,0.000000,0.442109
21,Samuel Adams Utopias,4.43,2021 vintage much more complex than my previou...,"[2021, vintage, much, complex, previous, vinta...",2021 vintage much complex previous vintages be...,0.00000,"{'neg': 0.0, 'neu': 0.775, 'pos': 0.225, 'comp...",0.4404,0.000000,0.506493


In [40]:
word_embed_sim_df = results_df[['product_name','user_rating',"cosine_similarity",'sentiment_score','word_embedding_similarity']].copy()
word_embed_rec = word_embed_sim_df.groupby(by='product_name').mean().sort_values(by='word_embedding_similarity', ascending=False)[:3].index.to_list()
word_embed_rec

['Oude Geuze Cuvée Armand & Gaston', 'Yellow Bus', 'BDCS']

In [41]:
word_embed_sim_df.groupby(by='product_name').mean().sort_values(by='word_embedding_similarity', ascending=False)[:3]

,user_rating,cosine_similarity,sentiment_score,word_embedding_similarity
product_name,,,,
Oude Geuze Cuvée Armand & Gaston,4.472857,0.080710,0.861404,0.682337
Yellow Bus,4.440000,0.065963,0.869188,0.665376
BDCS,4.395556,0.123605,0.839330,0.662176


In [42]:
def check_presence(text, attribute):
    x = text.find(attribute)
    if x==-1:
        y = 0
    else:
        y=1
    return y

In [43]:
bagofwords_rec = beer_reco_similarity['product_name'][:3].to_list()
bagofwords_rec

['Darkness', 'BDCS', 'Everett']

##### Calculating percentage of reviews that contain the attributes for bag of words recommended beers

In [44]:
df_bagofwords = results_df.loc[results_df['product_name'].isin(beer_reco_similarity['product_name'][:3]),['product_name','Cleaned_Sentence']].copy()
col_1 = ['product_name']
col_1.extend(selected_attributes)
bow_attr_per_rev = pd.DataFrame(columns=col_1)
bow_attr_per_rev['product_name'] = bagofwords_rec

for attribute in selected_attributes:
    df_bagofwords[attribute] = df_bagofwords['Cleaned_Sentence'].apply(lambda x: check_presence(x, attribute))

df_bagofwords_2 = df_bagofwords[col_1].groupby(by='product_name').aggregate(["sum","count"]).reset_index()

for attribute in selected_attributes:
    bow_attr_per_rev[attribute] = df_bagofwords_2[attribute]['sum']/df_bagofwords_2['dark']['count']

bow_attr_per_rev

,product_name,light,dark,carbonation
0,Darkness,0.555556,0.703704,0.370370
1,BDCS,0.459459,0.648649,0.324324
2,Everett,0.684211,0.421053,0.526316


##### Calculating percentage of reviews that contain the attributes for word embedding recommended beers

In [45]:
df_wordembed = results_df.loc[results_df['product_name'].isin(word_embed_rec),['product_name','Cleaned_Sentence']].copy()
col_2 = ['product_name']
col_2.extend(selected_attributes)
we_attr_per_rev = pd.DataFrame(columns=col_2)
we_attr_per_rev['product_name'] = word_embed_rec

for attribute in selected_attributes:
    df_wordembed[attribute] = df_wordembed['Cleaned_Sentence'].apply(lambda x: check_presence(x, attribute))
df_wordembed_2 = df_wordembed[col_2].groupby(by='product_name').aggregate(["sum","count"]).reset_index()

for attribute in selected_attributes:
    we_attr_per_rev[attribute] = df_wordembed_2[attribute]['sum']/df_wordembed_2['dark']['count']

we_attr_per_rev

,product_name,light,dark,carbonation
0,Oude Geuze Cuvée Armand & Gaston,0.555556,0.703704,0.370370
1,Yellow Bus,0.678571,0.071429,0.571429
2,BDCS,0.562500,0.125000,0.437500


The recommendations would significantly change if word vectors were utilized instead of the plain vanilla bag-of-words cosine similarity, and this difference is clearly highlighted when considering the percentage of reviews mentioning a preferred attribute. While the recommendations are Darkness, BDCS, and Everett for the Bag-of-Words approach, the recommendations shift to Oude Geuze Cuvée Armand & Gaston, Yellow Bus, and BDCS with word vectors. 

From these specific results, one can deduce that the comparison between bag-of-words and word vectors depends on the attribute being examined. With certain attributes, the percentage of reviews which state a preferred attribute is greater for the bag-of-words approach, while that same percentage is greater for word vectors for other attributes. This only further corroborates the content in the article - There are clear differences between the two approaches, but bag-of-words is not always better than word embeddings and vice versa. When it comes to some attributes, word embeddings “[do] wonders,” but in other cases, such as when “two or more words, which have distinct meanings, are likely to be mentioned close to each other,” word embeddings may not be as effective, and bag-of-words may perform comparatively better. Ultimately, the effect of each approach is dependent upon the attributes.

## Task G

How would your recommendations differ if you ignored the similarity and feature sentiment scores and simply chose the 3 highest rated products from your entire dataset? Would these products meet the requirements of the user looking for recommendations? Why or why not? Justify your answer with analysis. Use the similarity and sentiment scores as well as overall ratings to answer this question.










</font>
</div>

In [46]:
# identifying the top 3 beers based on customer rating
top_3 = df.groupby('product_name').agg({"user_rating":'mean'}).reset_index().sort_values(by = 'user_rating', ascending=False).head(3)
top_3

,product_name,user_rating
107,Oude Fermier,4.766190
30,Chemtrailmix - Rye Barrel,4.742353
35,Clover,4.729444


If we simply select the three highest rated beers, the top recommended beers are:
- Oude Fermier
- Chemtrailmix - Rye Barrel
- Clover

These completely differ from the beer recommendation we got considering similarity and sentiment scores. 
To see if these recommendations meet the requirements of the user, we calculated the similarity and sentiment scores of these beers.

In [47]:
results_df[results_df['product_name'].isin(top_3['product_name'])].groupby('product_name')['word_embedding_similarity','cosine_similarity','sentiment_score','eval_score'].mean()

,word_embedding_similarity,cosine_similarity,sentiment_score,eval_score
product_name,,,,
Chemtrailmix - Rye Barrel,0.484953,0.027360,0.611976,0.020700
Clover,0.560669,0.063302,0.696772,0.036373
Oude Fermier,0.604268,0.046403,0.865714,0.039824


Given the low similarity and sentiment scores, it would be unwise to suggest these products to customers. The bag-of-words similarity scores are notably minimal, and when compared to other beers, the overall similarity scores are not significantly high.
This indicates a lack of alignment between these beers and the attributes specified by the customer. In summary, placing reliance solely on customer ratings for recommendations could lead to misguidance when accounting for customer preferences.

This is probably due to the fact that similarity and sentiment scores offer a more thorough analysis. Sentiment scores allow for effective scrutiny of customer data to inform rational and sound business decisions. Moreover, numerical analyses like these minimize the potential for human error and misinterpretations. Similarly, similarity scores take into account both the quantity of data attributes and their variations, providing a more nuanced data analysis.
Simply relying on customer ratings, instead of assessing similarity and sentiment scores, represents a somewhat simplistic and restricted approach.


## Task - H

Using the top four attributes of beer (from word frequency analysis), calculate the lifts between these attributes and any 10 beers in your data. Choose one beer, and find the most similar beer (among the remaining 9) using the lift values. Explain your method. 

In [48]:
top_4_attributes = ['light', 'dark', 'carbonation', 'smooth']

In [49]:
top_10_beer = df.groupby('product_name')['product_name'].count().sort_values(ascending=False)[:10]
top_10_beer = top_10_beer.index.to_list()
top_10_beer

['Trappist Westvleteren 12 (XII)',
 'Bourbon County Brand Stout',
 'Focal Banger',
 'Bourbon County Brand Coffee Stout',
 'Parabola',
 'Darkness',
 'The Rusty Nail',
 'Sip Of Sunshine',
 'Zombie Dust',
 'Heady Topper']

In [50]:
df_sim = results_df.copy()

def lw(y):
    comment_tokens = word_tokenize(y)
    new_comment = []

    for i in comment_tokens:
        new_comment.append(i)

    return new_comment

df_sim['new_comment_tokens'] = df_sim['Cleaned_Sentence'].apply(lw)

def lift(n, a, b, ab):
    l = ((n*ab)/(a*b))
    return (l)

lift_db = df_sim.copy()
lift_db = lift_db[['product_name','Cleaned_Sentence','new_comment_tokens']].apply(pd.Series.explode).set_index(['product_name','Cleaned_Sentence']).reset_index().drop_duplicates().copy()

lift_values = pd.DataFrame(columns=['Beers','Attributes','lift_val'])

for x1 in top_10_beer:
    for x2 in top_4_attributes :
        lift_db['x1'] = 0
        lift_db['x2'] = 0

        lift_db['x1'][lift_db['product_name'] == x1] = 1
        lift_db['x2'][lift_db['new_comment_tokens'] == x2] = 1

        c = lift_db.groupby(['product_name','Cleaned_Sentence'])[['x1','x2']].sum().reset_index()

        a = lift_db[lift_db['product_name']==x1]['Cleaned_Sentence'].drop_duplicates().count()
        b = lift_db['x2'].sum()
        ab = c[(c['x1']>0) & (c['x2']==1)]['x2'].count()
        n = results_df['Cleaned_Sentence'].count()

        lift_val = lift(n, a, b, ab)

        app_dict = {}
        app_dict['Beers'] = x1
        app_dict['Attributes'] = x2
        app_dict['lift_val'] = lift_val

        lift_values = lift_values.append(app_dict, ignore_index=True)


In [51]:
similarity_table = pd.crosstab(lift_values['Beers'], lift_values['Attributes'], lift_values['lift_val'], aggfunc=np.mean)
similarity_table

Attributes,carbonation,dark,light,smooth
Beers,,,,
Bourbon County Brand Coffee Stout,1.585682,2.428844,1.232267,1.837630
Bourbon County Brand Stout,0.773504,1.545394,0.721327,1.536694
Darkness,1.285688,2.625777,1.198963,1.419019
Focal Banger,0.991051,0.000000,0.616134,1.312593
Heady Topper,0.660701,0.234671,0.273837,1.166749
Parabola,1.118109,2.816051,0.505545,1.884748
Sip Of Sunshine,0.440467,0.117335,0.547674,0.875062
The Rusty Nail,1.821392,2.054956,1.198963,1.419019
Trappist Westvleteren 12 (XII),1.038244,1.508599,1.525664,1.125079


In [52]:
random_beer = similarity_table.loc['Bourbon County Brand Coffee Stout']
random_beer

Attributes
carbonation    1.585682
dark           2.428844
light          1.232267
smooth         1.837630
Name: Bourbon County Brand Coffee Stout, dtype: float64

In [53]:
difference_table = np.abs(similarity_table[:].values-random_beer.values)
sum_array = np.sum(difference_table, axis=1)
min_ind = np.argmin(sum_array, axis = None, out=None)
sum_array = np.delete(sum_array,min_ind,axis = 0)
min_ind = np.argmin(sum_array, axis = None, out = None)

In [55]:
similar_beer = similarity_table[min_ind+1:min_ind+2].index.values
similar_beer

array(['Darkness'], dtype=object)

Our analysis focused on identifying key attributes through word frequency analysis, resulting in the identification of the top four attributes: light, dark, carbonation, and smooth. Subsequently, we selected the top 10 beers and assessed their lift values with each of these attributes.

To identify similar beers, we randomly chose 'Bourbon County Brand Coffee Stout' and meticulously computed the difference in its lift values with the remaining 9 beers in our selection.

This analysis revealed a noteworthy similarity between 'Bourbon County Brand Coffee Stout' and 'Darkness.' The lift values for both beers exceeded 1, indicating a positive association between their attribute occurrences. Specifically, 'Bourbon County Brand Coffee Stout' and 'Darkness' share similarities in being dark and smooth, contributing to their elevated lift values. This observation suggests that these two beers exhibit a closer relationship in terms of the identified attributes, reinforcing the notion that they are more alike in certain aspects, particularly in their darkness and smoothness.